## Experimentation with Different Models


## Introduction to YOLOv8 for Object Detection in Inclement Weather

Object detection in challenging conditions, such as inclement weather, requires a robust and efficient model capable of recognizing objects with high precision despite visual impairments caused by rain, fog, snow, or low light. The YOLO (You Only Look Once) series has been at the forefront of object detection tasks due to its speed and accuracy, and the latest iteration, YOLOv8, brings further improvements in both performance and efficiency.

YOLOv8 stands out for its ability to process images in real-time with impressive accuracy, making it an ideal candidate for applications requiring quick and reliable object detection in adverse weather conditions. Its architecture is designed to handle the complexities and variability of real-world scenarios, which is critical for ensuring safety and operational efficiency in environments affected by inclement weather.

We will experiment with the three available variants of the YOLOv8 model: nano, small, and medium. These models vary in size and computational requirements, allowing us to assess which variant offers the best balance between speed and accuracy for our prototype. The **nano** model, being the lightest, promises the fastest inference times but at potentially lower accuracy. The **small** model offers a middle ground, aiming for a balance between performance and speed. Lastly, the **medium** model provides the highest accuracy at the cost of increased computational load. Our goal is to identify the most suitable model for object detection in inclement weather, ensuring that our prototype can operate effectively under challenging conditions.


## 1. Data Label Fromat

### 1.1. Original Annotation Format to KITTI Format

The filtermoosekitti.py script, sourced from the asvath/cadcd repository, plays a crucial role in ensuring the integrity and standardization of our dataset annotations. Developed by the original author, this script filters annotations based on a minimum of 20 lidar points, effectively eliminating potential noise or outliers from the ground truth data. This methodical approach not only aligns annotations with the KITTI format but also incorporates essential details such as truncation, observation angle, and pixel coordinates. The script's commitment to maintaining a standardized layout enhances the overall robustness of the dataset, making it well-suited for experimentation with various object detection models. For further reference, you can find the script in the asvath/cadcd repository.

Then, we rename the images inside each Image_## folder in the 'processedmoose' so that the images' names matches the annotation files' numenclature.


In [ ]:
import os

# Define the parent directory where the 'image_00', 'image_01', ..., 'image_07' folders are located
parent_directory = 'C:\\Users\\rafae\\Documents\\Capstone\\autonomoose\\processedmoose'  

# Loop through each subfolder (image_00, image_01, ..., image_07)
for i in range(8):
    subfolder_name = 'image_{:02d}'.format(i)
    subfolder_path = os.path.join(parent_directory, subfolder_name, 'data') 

    # Loop through each image in the subfolder
    for j in range(98):
        old_image_name = '000000{:04d}.png'.format(j)
        new_image_name = '000000{:04d}.png'.format(i * 100 + j)

        old_image_path = os.path.join(subfolder_path, old_image_name)
        new_image_path = os.path.join(subfolder_path, new_image_name)

        # Rename the image
        os.rename(old_image_path, new_image_path)

Great, we have remaned all images in the folers image_##: {old_image_name} -> {new_image_name}
    
Image_00 > 0000000002.png -> 0000000002.png, ..., 00000000099.png -> 00000000099.png
    
Image_01 > 0000000002.png -> 0000000102.png, ..., 00000000099.png -> 00000000199.png
    
...
    
Image_07 > 0000000002.png -> 0000000702.png, ..., 00000000099.png -> 00000000799.png

### 1.2. KITTI to VOC Dataset Converter

Now, we can transform the the data from KITTI format to VOC format using the Visual Object Dataset Converter from https://github.com/umautobots/vod-converter/blob/master/README.md 

Run the following command in the terminal to convert datasets from the KITTI format to VOC:

```bash
python vod-converter/vod_converter/main.py --from kitti --from-path cadc_data --to voc --to-path mydata-voc


### 1.3. Dataset Preprocessing with Roboflow

Now that all images share the same names as their corresponding annotation files, we are ready to upload this dataset to Roboflow. Roboflow plays a crucial role in facilitating the data formatting step. With the images and annotations in VOC format, Roboflow can read this format and convert it into other widely used standards, such as YOLO. This eliminates the need for manual coding during format changes. Also, Roboflow also has augmentation settings that we can use when processing the dataset.

We successfully uploaded the 198 images along with their corresponding annotation files to Roboflow. During this process, Roboflow automatically filtered out any annotations with invalid data, such as negative height or width, empty annotations, and more. As a result, we now have a dataset containing 198 annotated images, 100 with no snow, 98 with snow-covered roads, featuring **three classes**: 

**1. Car**

**2. Truck**

**3. Person** (*Pedestrian* was renamed to *Person* since we're using the pretrained model on COCO dataset which has the class *Person* not *Pedestrian*)

Subsequently, Roboflow partitioned the dataset into training, validation, and test sets.

## 2. YOLOv8 

### 2.1. Experiment Design and Model Selection for Cloudy and Snowy Conditions

In our endeavor to develop a robust object detection prototype, we are conducting an experiment using a dataset that initially comprises 100 images captured under cloudy weather conditions. Recognizing the importance of versatility in real-world applications, we plan to augment this dataset with an additional 98 images featuring snow conditions. This expansion will enable us to assess the prototype's performance across a broader spectrum of inclement weather scenarios. Our experiment is structured to run for 50 epochs, providing a thorough training period for the models under consideration.

### Dataset Composition Analysis
The initial focus on cloudy weather aims to address the specific challenges posed by overcast skies and diminished lighting. The subsequent inclusion of snow images is strategic, introducing complexities such as occlusion and altered landscape contrast, which are critical for testing the model's adaptability and accuracy in diverse environments. 

Aditionally, the dataset encompasses a total of 720 labels, distributed across two categories:

- **Cars**: 599 labels, the majority of our dataset, focusing on vehicular detection.
- **Trucks**: 121 labels, introducing larger vehicle detection challenges.

The modest size of our dataset, while potentially limiting, is intentionally chosen to ensure focused, rapid initial experimentation. We acknowledge the risks associated with a small dataset, such as overfitting, and plan to implement data augmentation and appropriate regularization techniques to mitigate these concerns. This dataset serves as a pivotal foundation for establishing baseline performance metrics and understanding the nuanced effects of weather on detection capabilities.

The dataset used in this notebook can be downloaded from Roboflow using the badge bellow.


<a href="https://universe.roboflow.com/machine-learning-engineer-bootcamp/object_detect-bh8cz">
    <img src="https://app.roboflow.com/images/download-dataset-badge.svg"></img>
</a>

### Benchmarking Strategy with YOLOv8 Variants
Our benchmarking strategy encompasses evaluating three variants of the YOLOv8 model: nano, small, and medium. This selection is guided by the need to balance computational efficiency with detection accuracy:

- **YOLOv8 nano**: Tested for its speed and efficiency, particularly valuable for real-time applications, albeit with an anticipated compromise in accuracy under complex weather conditions.
- **YOLOv8 small**: Expected to strike an optimal balance, potentially offering superior accuracy to the nano model with manageable computational demands, making it a strong candidate for the prototype.
- **YOLOv8 medium**: While the most computationally intensive, it will be benchmarked for its maximum achievable accuracy, setting an aspirational standard for detection performance in adverse weather.

### Future Directions and Model Choice
The inclusion of snow images and the evaluation on the chosen YOLOv8 model, together with fine-tuning, will inform our decision on the most suitable model for our prototype. The aim is to select a model that demonstrates high accuracy in diverse weather conditions while maintaining computational efficiency. 

This initial phase of experimentation and model selection lays the groundwork for subsequent prototype iterations. With the chosen model, we intend to further refine our approach by expanding the dataset, incorporating more varied weather conditions, and optimizing model parameters to enhance detection capabilities in real-world scenarios. The insights gained will drive the continuous improvement of our object detection solution, ensuring reliability and effectiveness in changing environmental conditions.

### 2.2. Experimenting with Different Models

In [8]:
from ultralytics import YOLO
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
Setup complete ✅ (4 CPUs, 15.8 GB RAM, 233.7/953.3 GB disk)


### YOLOv8n (Nano)

In [9]:
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

In [10]:
# Train the model
results = model.train(data='data.yaml', epochs=50, imgsz=640, plots=True)

New https://pypi.org/project/ultralytics/8.1.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

train: Scanning C:\Users\rafae\Documents\Capstone\object_detect-1\train\labels.cache... 63 images, 11 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]
val: Scanning C:\Users\rafae\Documents\Capstone\object_detect-1\valid\labels.cache... 19 images, 1 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]

Plotting labels to runs\detect\train7\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      2.277      4.127      1.452        121        640: 100%|██████████| 4/4 [01:05<00:00, 16.46s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:09<00:00,  9.05s/it]

...

      50/50         0G       1.22      1.234      1.054         97        640: 100%|██████████| 4/4 [01:01<00:00, 15.39s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:08<00:00,  8.89s/it]

                   all         19        151      0.561      0.591      0.621      0.379



50 epochs completed in 1.145 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 6.3MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:08<00:00,  8.17s/it]


                   all         19        151      0.556      0.581      0.621      0.379
                 truck         19         26       0.46      0.394      0.471      0.275
                   car         19        125      0.652      0.768      0.771      0.482
Speed: 5.8ms preprocess, 357.6ms inference, 0.0ms loss, 50.3ms postprocess per image
Results saved to runs\detect\train7


In [12]:
# Train the model
#results = model.train(data='data.yaml', epochs=50, imgsz=640, plots=True)

metrics = model.val()
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\rafae\Documents\Capstone\object_detect-1\valid\labels.cache... 19 images, 1 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:07<00:00,  3.62s/it]


                   all         19        151      0.556      0.581      0.621      0.379
                 truck         19         26       0.46      0.394      0.471      0.275
                   car         19        125      0.652      0.768      0.771      0.482
Speed: 10.0ms preprocess, 316.2ms inference, 0.0ms loss, 34.9ms postprocess per image
Results saved to runs\detect\train72


array([      0.275,     0.48229])

![Description or alt text for the image](runs\detect\train7\results.png "Optional title")

### YOLOv8s (Small)

In [4]:
model1 = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)
# Train the model
results = model1.train(data='data.yaml', epochs=50, imgsz=640)

metrics = model1.val()
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

New https://pypi.org/project/ultralytics/8.1.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

train: Scanning C:\Users\rafae\Documents\Capstone\datasets\train\labels.cache... 63 images, 11 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]
val: Scanning C:\Users\rafae\Documents\Capstone\datasets\valid\labels.cache... 19 images, 1 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]


Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      2.293      3.982      1.472        121        640: 100%|██████████| 4/4 [02:32<00:00, 38.10s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:16<00:00, 16.88s/it]

                   all         19        151      0.424       0.26      0.278      0.134


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:14<00:00, 14.60s/it]


                   all         19        151      0.759      0.792      0.806      0.542
                 Truck         19         26      0.612      0.808      0.775      0.488
                   car         19        125      0.906      0.776      0.838      0.596
Speed: 5.8ms preprocess, 742.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train4
Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\rafae\Documents\Capstone\datasets\valid\labels.cache... 19 images, 1 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:14<00:00,  7.22s/it]


                   all         19        151      0.759      0.792      0.806      0.542
                 Truck         19         26      0.612      0.808      0.775      0.488
                   car         19        125      0.906      0.776      0.838      0.596
Speed: 5.8ms preprocess, 739.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train42


array([    0.48753,     0.59627])

![Description or alt text for the image](runs\detect\train4\results.png "Optional title")

### YOLOv8m (Medium)

In [6]:
model2 = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)
# Train the model
results = model2.train(data='data.yaml', epochs=50, imgsz=640)

metrics = model2.val()
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

New https://pypi.org/project/ultralytics/8.1.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

train: Scanning C:\Users\rafae\Documents\Capstone\object_detect-1\train\labels... 63 images, 11 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<00:00, 122.25it/s]

train: New cache created: C:\Users\rafae\Documents\Capstone\object_detect-1\train\labels.cache



val: Scanning C:\Users\rafae\Documents\Capstone\object_detect-1\valid\labels... 19 images, 1 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<00:00, 107.12it/s]

val: New cache created: C:\Users\rafae\Documents\Capstone\object_detect-1\valid\labels.cache


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:35<00:00, 35.92s/it]


                   all         19        151      0.869       0.82      0.887      0.632
                 Truck         19         26      0.844       0.83      0.902      0.656
                   car         19        125      0.894      0.811      0.873      0.607
Speed: 5.8ms preprocess, 1865.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train6
Ultralytics YOLOv8.0.225 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\rafae\Documents\Capstone\object_detect-1\valid\labels.cache... 19 images, 1 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:35<00:00, 17.85s/it]


                   all         19        151      0.869       0.82      0.887      0.632
                 Truck         19         26      0.844       0.83      0.902      0.656
                   car         19        125      0.894      0.811      0.873      0.607
Speed: 6.6ms preprocess, 1855.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train62


array([    0.65557,     0.60745])

![Description or alt text for the image](runs\detect\train6\results.png "Optional title")


## 2.3 Models Comparsion

![Description or alt text for the image](yolov8.png "Optional title")

Training Time:

    -Nano: 1.145 hours
    -Small: 2.220 hours
    -Medium: 5.668 hours
    
Based on the training results and the given table for YOLOv8n (nano), YOLOv8s (small), and YOLOv8m (medium) models, here are some comparisons and considerations, especially in the context of a small dataset of only 100 images:

### Model Performance

**Accuracy:** There's a clear trend where the size of the model correlates with accuracy. The medium model has the highest mAP, followed by the small model, and then the nano model. However, the difference in mAP between the small and medium models is not as pronounced as between nano and small.

**Training Time:** As expected, the larger the model, the longer the training time. The nano model is the fastest to train, with the medium taking significantly longer.

### Loss and Metrics Trends

The training and validation loss for all models decrease over time, which is a good sign of learning. However, the larger models (small and medium) tend to have smoother and lower loss curves, indicating better performance.
The precision and recall metrics improve over time for all models. However, larger models tend to have higher precision and recall, indicating better overall predictive performance.

Larger models show higher mAP scores across both the 0.5 and 0.5-0.95 IoU thresholds, suggesting they are better at both detecting objects (mAP@0.5) and accurately predicting their shapes and sizes (mAP@0.5-0.95).

### Computational Resources and Time Constraints

**Nano Model:** Best if you have very strict time constraints and limited computational resources. It offers the least accuracy but is the fastest to train.

**Small Model:** Presents a balanced choice between accuracy and training time. It requires more time than the nano model but is significantly faster than the medium model, making it a suitable option if you have moderate resources.

**Medium Model:** While it offers the best accuracy, the training time is considerably longer. It may not be the best choice if you have limited computational resources or if the training time is a critical factor.

### Considering a Small Dataset

With a small dataset of 100 images, the risk of overfitting increases, especially with larger models. Regularization and data augmentation become more important to prevent this.
The benefit of larger models may not be as significant with a small dataset since they're designed to capture more complex patterns that may not be present in limited data.

## 2.4 Final Consideration

Given the limited dataset size and considering the balance between training time and accuracy, the small model (YOLOv8s) seems to be a good middle ground. It offers a substantial improvement in accuracy over the nano model for a reasonable increase in training time. Since the accuracy of the small and medium models is relatively close, the doubled training time for the medium model may not be justifiable, especially under limited computational resources and time constraints.

It's also worth noting that the increase in training time is not linear with the increase in model size. The medium model takes over twice as long to train as the small model, which doesn't correspond to a doubling in accuracy, indicating diminishing returns for the increased computational cost.

In summary, for your scenario, the YOLOv8s model would likely be the best option, offering a good trade-off between the increased accuracy it provides over the nano model and the significantly lower training time compared to the medium model.

## 3. Possible Model Fine-tuning Strategies

Based on the training results for the YOLOv8n (nano), YOLOv8s (small), and YOLOv8m (medium) models, we have the following recommendations for prototype fine-tuning:

### Dataset Expansion
- **Expand the dataset**: More images will help with generalization and reduce the risk of overfitting.

### Data Augmentation
- **Enhance data augmentation**: Introduce more variability to make the model robust, especially with smaller datasets.

### Regularization Techniques
- **Implement regularization**: Use techniques like dropout or weight decay to prevent overfitting.

### Hyperparameter Tuning
- **Optimize hyperparameters**: Systematically tune learning rate, batch size, and others, possibly using automated tools like Optuna or Hyperopt.

### Early Stopping and Patience
- **Early stopping**: Implement early stopping with a patience value (e.g., ~30 epochs) and adjust based on observed training behavior.

### Learning Rate Scheduling
- **Adjust learning rate schedule**: Experiment with reducing the learning rate on a plateau or cyclical learning rates.


In summary, the YOLOv8s model provides a balance between training efficiency and accuracy improvement over the nano model. As we scale our dataset and continue to fine-tune the model, maintaining a balance between model complexity and dataset size will be crucial.


## References

For further details and resources, please refer to the following:

- Ultralytics YOLO Models and Utilities: [Ultralytics GitHub](https://github.com/ultralytics/ultralytics)
- Canadian Adverse Driving Conditions Dataset: [CADC GitHub](https://github.com/asvath/cadcd)
- Video Object Detection Converter: [VOD-Converter GitHub](https://github.com/umautobots/vod-converter/blob/master/README.md)
- Getting Started with Training a YOLOv8 Object Detection Model: [Datature Blog](https://www.datature.io/blog/get-started-with-training-a-yolov8-object-detection-model)
